In [31]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist_m as mnistm
from data import mnist
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from data.tasks import load_task
from experiments.training import *
from experiments.SL_bound import *
from experiments.DA_bound import *
from bounds.bounds import *
from util.kl import *
from util.misc import *
from util.batch import *
from results.plotting import *

# Hyper-parameters
task = 2
seed = 69105
n_classifiers = 20
delta=0.05 ## what would this be?   
binary=True
bound='germain'
epsilons=[0.01]
alphas=[0.1]#0,0.3]
sigmas=[[3,2],[1,2],[3,3]]

project_folder = "/cephyr/users/frejohk/Alvis/projects/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Alvis params

In [27]:
project = 'SNIC2021-7-82'
username = 'frejohk'
job = 'batch_bound_single.sbatch'

In [28]:
!squeue -u $username

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
            103465     alvis     bash  frejohk  R    6:04:33      1 alvis1-08 


In [29]:
!cat $job

#!/usr/bin/bash

#SBATCH -t 1-00:00:00
#SBATCH -N 1 --gpus-per-node=T4:1
#SBATCH -p alvis 

if [ -z "$task" ]
then
    task=2
fi
if [ -z "$seed" ]
then 
    seed=69105
fi
if [ -z "$alpha" ]
then 
    alpha=0.0
fi
if [ -z "$sigma" ]
then 
    sigma='3,3'
fi
if [ -z "$epsilon" ]
then 
    epsilon=0.01
fi
if [ -z "$delta" ]
then 
    delta=0.05
fi
if [ -z "$binary" ]
then 
    binary=0
fi
if [ -z "$nclassifiers" ]
then 
    nclassifiers=2
fi
if [ -z "$bound" ]
then 
    bound='germain'
fi
if [ -z "$prior" ]
then 
    prior=''
fi
if [ -z "$posterior" ]
then 
    posterior=''
fi

. load_modules.sh

python batch_bound_single.py -t $task -r $seed -a $alpha -s $sigma -e $epsilon -d $delta -b $binary -n $nclassifiers -B $bound -p $prior -P $posterior

In [37]:
print('Iterating over experiments...\n')

fids = []

os.makedirs('logs', exist_ok=True)


np.random.seed(seed)
for alpha in alphas:
    
    print("alpha:"+str(alpha))
    
    for epsilon in epsilons:
        print("  epsilon:"+str(epsilon))
        for sigma in sigmas:    
            print("    sigma:"+str(sigma))
            arg_list = get_job_args(task, bound=bound, alpha=alpha, sigma=sigma,
                                    epsilon=epsilon, binary=binary, n_classifiers=n_classifiers)
            for a in arg_list:   
                
                ckpt = os.path.splitext(os.path.basename(a['posterior_path']))[0]
                fid = './logs/batch_t-%d_r-%d_a-%.4f_s-%d%d_e-%.4f_d-%.4f_b-%d_n-%d_B-%s_c-%s' % \
                    (task, seed, alpha, sigma[0], sigma[1], epsilon, delta, binary, n_classifiers, bound, ckpt)
                
                sigstr = '"%d.%d"' % (sigma[0], sigma[1])
                exp = 'task=%d,seed=%d,alpha=%.4f,sigma="%s",epsilon=%.4f,delta=%.4f,'% (task, seed, alpha, sigstr, epsilon, delta)\
                        +'binary=%d,nclassifiers=%d,bound=%s,prior=%s,posterior=%s' % (binary, n_classifiers, bound, a['prior_path'], a['posterior_path'])
                
                prior_path = a['prior_path']
                output = !sbatch -o "$fid"-%j.out -e "$fid"-%j.err -A $project --export="$exp" $job

                jobid = int(output[0].split(' ')[-1])
                fid = fid+'-%s' % jobid

                fids.append(fid)

Iterating over experiments...

alpha:0.1
  epsilon:0.01
    sigma:[1, 2]


In [36]:
print(fids)

['./logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_0-103884', './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_45-103885', './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_90-103886', './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_135-103887', './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_180-103888', './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_225-103889', './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_270-103890', './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_315-103891', './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_360-103892', './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_405-103893', './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n

In [44]:
!squeue -u $username

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
            103465     alvis     bash  frejohk  R    6:11:43      1 alvis1-08 


In [45]:
for fid in fids: 
    print('------')
    print(fid)
    !tail -n 50 "$fid".out
    !tail -n 5 "$fid".err
    print(' \n\n')

------
./logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_0-103913
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=103913&from=1632840833000
2021-09-28 16:53:56.750284: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
slurmstepd: error: *** JOB 103913 ON alvis2-19 CANCELLED AT 2021-09-28T16:54:04 ***
 


------
./logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_45-103914
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=103914&from=1632840834000
2021-09-28 16:53:57.608219: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
slurmstepd: error: *** JOB 103914 ON alvis2-19 CANCELLED AT 2021-09-28T16:54:07 ***
 


------
./logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-1_90-103

tail: cannot open './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-2_6-103931.err' for reading: No such file or directory
 


------
./logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-2_7-103932
tail: cannot open './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-2_7-103932.out' for reading: No such file or directory
tail: cannot open './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-2_7-103932.err' for reading: No such file or directory
 


------
./logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-2_8-103933
tail: cannot open './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-2_8-103933.out' for reading: No such file or directory
tail: cannot open './logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-10_B-germain_c-2_8-103933.err' for reading: No such file or directory
 


------
./logs/batch_t-2_r-69105_a-0.